In [1]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
# -*- coding: utf-8 -*-

# TensowFlowのインポート
import tensorflow as tf
# MNISTを読み込むためinput_data.pyを同じディレクトリに置きインポートする
# input_data.pyはチュートリアル内にリンクがあるのでそこから取得する
# https://tensorflow.googlesource.com/tensorflow/+/master/tensorflow/examples/tutorials/mnist/input_data.py
import input_data

import time

# 開始時刻
start_time = time.time()
print "開始時刻: " + str(start_time)

# MNISTデータの読み込み
# 60000点の訓練データ（mnist.train）と10000点のテストデータ（mnist.test）がある
# 訓練データとテストデータにはそれぞれ0-9の画像とそれに対応するラベル（0-9）がある
# 画像は28x28px(=784)のサイズ
# mnist.train.imagesは[60000, 784]の配列であり、mnist.train.lablesは[60000, 10]の配列
# lablesの配列は、対応するimagesの画像が3の数字であるならば、[0,0,0,1,0,0,0,0,0,0]となっている
# mnist.test.imagesは[10000, 784]の配列であり、mnist.test.lablesは[10000, 10]の配列
print "--- MNISTデータの読み込み開始 ---"
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print "--- MNISTデータの読み込み完了 ---"

# 訓練画像を入れる変数
# 訓練画像は28x28pxであり、これらを1行784列のベクトルに並び替え格納する
# Noneとなっているのは訓練画像がいくつでも入れられるようにするため
x = tf.placeholder(tf.float32, [None, 784])

# 重み
# 訓練画像のpx数の行、ラベル（0-9の数字の個数）数の列の行列
# 初期値として0を入れておく
W = tf.Variable(tf.zeros([784, 10]))

# バイアス
# ラベル数の列の行列
# 初期値として0を入れておく
b = tf.Variable(tf.zeros([10]))

# ソフトマックス回帰を実行
# yは入力x（画像）に対しそれがある数字である確率の分布
# matmul関数で行列xとWの掛け算を行った後、bを加算する。
# yは[1, 10]の行列
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 交差エントロピー
# y_は正解データのラベル
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# 勾配硬化法を用い交差エントロピーが最小となるようyを最適化する
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# 用意した変数Veriableの初期化を実行する
init = tf.initialize_all_variables()

# Sessionを開始する
# runすることで初めて実行開始される（run(init)しないとinitが実行されない）

sess = tf.Session()
sess.run(init)

# 1000回の訓練（train_step）を実行する
# next_batch(100)で100つのランダムな訓練セット（画像と対応するラベル）を選択する
# 訓練データは60000点あるので全て使いたいところだが費用つまり時間がかかるのでランダムな100つを使う
# 100つでも同じような結果を得ることができる
# feed_dictでplaceholderに値を入力することができる
print "--- 訓練開始 ---"
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})
print "--- 訓練終了 ---"

# 正しいかの予測
# 計算された画像がどの数字であるかの予測yと正解ラベルy_を比較する
# 同じ値であればTrueが返される
# argmaxは配列の中で一番値の大きい箇所のindexが返される
# 一番値が大きいindexということは、それがその数字である確率が一番大きいということ
# Trueが返ってくるということは訓練した結果と回答が同じということ
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# 精度の計算
# correct_predictionはbooleanなのでfloatにキャストし、平均値を計算する
# Trueならば1、Falseならば0に変換される
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# 精度の実行と表示
# テストデータの画像とラベルで精度を確認する
# ソフトマックス回帰によってWとbの値が計算されているので、xを入力することでyが計算できる
print "精度"
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

# 終了時刻
end_time = time.time()
print "終了時刻: " + str(end_time)
print "かかった時間: " + str(end_time - start_time)

ImportError: No module named input_data

In [ ]:
'''

'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import os.path
import numpy
import matplotlib.pyplot as plt

# dimensions of our images.
img_width, img_height = 56, 56

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

f_log = './log'
#model saving location
f_model = './model'
model_filename = 'cnn_model.json'
# weights_filename = 'cnn_model_weights.hdf5' 
weights_filename = 'weights.hdf5'

train_sample_number = 56
test_sample_number = 41

nb_train_samples = train_sample_number*30
nb_validation_samples = test_sample_number*20
train_batch = train_sample_number
test_batch = test_sample_number
nb_epoch = 20


#training model
model = Sequential()

model.add(Convolution2D(32, 3, 3,border_mode='valid', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Convolution2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Convolution2D(128, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Convolution2D(128, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

sgd = SGD(lr=5e-3, decay=1e-6, momentum=0.9, nesterov=True)
    
model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['accuracy'])


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    zca_whitening=True,
    zoom_range=0.2,
    rotation_range=180, 
    width_shift_range=0.3,
	height_shift_range=0.3,
	fill_mode='nearest',
	vertical_flip=True,
    horizontal_flip=True
)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(

    zoom_range=0.2,
    rotation_range=180, 
    vertical_flip=True,
    horizontal_flip=True
    )


train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=train_batch,

	# save_to_dir="saved",   
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=test_batch,
        class_mode='binary')

checkpointer = ModelCheckpoint(filepath="./saved/weights.hdf5", verbose=1, save_best_only=True)

history = model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples,
        callbacks=[checkpointer])


print('save the architecture of a model')
json_string = model.to_json()
open(os.path.join(f_model,'cnn_model.json'), 'w').write(json_string)
yaml_string = model.to_yaml()
open(os.path.join(f_model,'cnn_model.yaml'), 'w').write(yaml_string)
print('save weights')
model.save_weights(os.path.join(f_model,'cnn_model_weights.hdf5'))

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Found 56 images belonging to 2 classes.
Found 41 images belonging to 2 classes.
Epoch 1/20
 728/1680 [============>.................] - ETA: 29s - loss: 0.1420 - acc: 0.7720

In [1]:
from keras.datasets import cifar10
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
# x_train shape: (50000, 32, 32, 3)
# y_train shape: (50000, 1)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Using TensorFlow backend.


('x_train shape:', (50000, 32, 32, 3))
('y_train shape:', (50000, 1))
(50000, 'train samples')
(10000, 'test samples')


In [ ]:
import os
from PIL import Image
import PIL.ImageOps
import urllib.parse
from itertools import count
import random


"""
    Turnes every image file to 28*28
    Also invert color
    """

iid = count()
def random_id(length):
    number = '0123456789'
    alpha = 'abcdefghijklmnopqrstuvwxyz'
    id = ''
    for i in range(0,length,2):
        id += random.choice(number)
        id += random.choice(alpha)
    return id

def resizer(cur_path):
    for a in os.listdir(cur_path):
        if(os.path.isdir('%s/%s'%(cur_path,a))):
            # resizer('%s/%s'%(cur_path,a))
            print('%s/%s'%(cur_path,str(a)))
        else:

            # img = Image.open('%s/%s'%(cur_path,a), 'r')
            # newNameTag= random_id(6)
            # newName= newNameTag + str(next(iid)) + ".png"
            # os.rename('%s/%s'%(cur_path,a),'%s/%s'%(cur_path,newName))
            try:
                img = Image.open('%s/%s'%(cur_path,a), 'r')
                newNameTag= random_id(6)
                newName= newNameTag + str(next(iid)) + ".png"
                os.rename('%s/%s'%(cur_path,a),'%s/%s'%(cur_path,newName))
                
            except:				
                print("file not image ",str(a))


resize_size=(56,56)
cur_path = os.path.dirname(os.path.abspath(__file__))
resizer(cur_path+"/data")

In [11]:
import cv2
import numpy as np
dir= "data/train/1/0f8j3b2.png"
dir1= "data/train/1/0k7o4v7.png"

big_array = []
im = cv2.imread(dir)
if im.shape[0] != 56 and im.shape[1] != 56:
    im = cv2.resize(im, (56, 56))
im = im.tolist()
nparray = np.asarray(im)
print(nparray.shape)
big_array.append(im)
print("big array=")
print(np.asarray(big_array).shape)



dir1= "data/train/1/0k7o4v7.png"

im1 = cv2.imread(dir1)
im1 = im1.tolist()
nparray = np.asarray(im1)
print(nparray.shape)
big_array.append(im1)

print("big array=")
print(np.asarray(big_array).shape)




(56, 56, 3)
big array=
(1, 56, 56, 3)
(56, 56, 3)
big array=
(2, 56, 56, 3)


In [24]:
def read_image_as_array(dir):
    """
    :param dir: directory of image name
    :return array
    """

    im = cv2.imread(dir)
    #     if im.shape[0] != 56 and im.shape[1] != 56:
    #         im = cv2.resize(im, (56, 56))
    im = cv2.resize(im, (56, 56))
    im = im.tolist()
    return im

dir= "data/train/1/0f8j3b2.png"
print(len(read_image_as_array(dir)))
print(len(read_image_as_array(".DS_store")))


56


error: /build/opencv-SviWsf/opencv-2.4.9.1+dfsg/modules/imgproc/src/imgwarp.cpp:1834: error: (-215) ssize.area() > 0 in function resize


In [22]:
import os
from PIL import Image
import PIL.ImageOps
# import urllib.parse # for python 3
from urlparse import urlparse # for python 2
from itertools import count
import random
import cv2
import numpy as np

"""
data/
    train/
        0/
            dog001.jpg
            dog002.jpg
            ...
        1/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        0/
            dog001.jpg
            dog002.jpg
            ...
        1/
            cat001.jpg
            cat002.jpg
            ...
"""

def make_label_for_each_directory(dir):
    """
    :param dir
    :return (big_array, label_array): (nparray, label)
    """

    big_array = []
    label_array = []
    for index, class_name in enumerate(os.listdir(dir)):
        abs_path_class_name = dir + "/" + class_name
        if(os.path.isdir(abs_path_class_name)):
            print("labeling class_name"+'%s/%s'%(dir,str(class_name)) + "as " '%s'%index)
            #appending index to label_array 
            labelname = index
            for pic_name in os.listdir(abs_path_class_name):
                abs_path_pic_name = abs_path_class_name + "/" + pic_name
                pic_array = read_image_as_array(abs_path_pic_name)
                big_array.append(pic_array)
                print(np.asarray(big_array).shape)
                label_array.append(labelname)
        else:
            print("not folder, %s" % class_name)
    #turn python array to numpy
    return (np.asarray(big_array), np.asarray(label_array))

def read_image_as_array(dir):
    """
    :param dir: directory of image name
    :return array
    """

    im = cv2.imread(dir)
    #     if im.shape[0] != 56 and im.shape[1] != 56:
    #         im = cv2.resize(im, (56, 56))
    im = cv2.resize(im, (56, 56))
    im = im.tolist()
    return im

def data_generate(cur_path):
    """
    :param cur_path: input directory of train, validation data 
    :return (x_train, y_train), (x_test, y_test)
    """
    (x_train, y_train) = make_label_for_each_directory(cur_path + "/train")
    (x_test, y_test) = make_label_for_each_directory(cur_path + "/validation")
    return (x_train, y_train), (x_test, y_test)

data_directory = "data"
cur_path = os.path.dirname(os.path.abspath('__file__'))
(x_train, y_train), (x_test, y_test) = data_generate(cur_path + "/" + data_directory)

# The data, shuffled and split between train and test sets:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
# x_train shape: (50000, 32, 32, 3)
# y_train shape: (50000, 1)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


# Reference
# http://stackoverflow.com/questions/29839350/numpy-append-vs-python-append

not folder, .DS_Store
labeling class_name/kerasTest/data/train/0as 1
(1, 56, 56, 3)
(2, 56, 56, 3)
(3, 56, 56, 3)
(4, 56, 56, 3)
(5, 56, 56, 3)
(6, 56, 56, 3)
(7, 56, 56, 3)
(8, 56, 56, 3)
(9, 56, 56, 3)
(10, 56, 56, 3)
(11, 56, 56, 3)
(12, 56, 56, 3)
(13, 56, 56, 3)
(14, 56, 56, 3)
(15, 56, 56, 3)
(16, 56, 56, 3)
(17, 56, 56, 3)
(18, 56, 56, 3)
(19, 56, 56, 3)
(20, 56, 56, 3)
(21, 56, 56, 3)
(22, 56, 56, 3)
(23, 56, 56, 3)
(24, 56, 56, 3)
(25, 56, 56, 3)
(26, 56, 56, 3)
(27, 56, 56, 3)
(28, 56, 56, 3)
(29, 56, 56, 3)
(30, 56, 56, 3)
(31, 56, 56, 3)
(32, 56, 56, 3)
(33, 56, 56, 3)
(34, 56, 56, 3)
(35, 56, 56, 3)
(36, 56, 56, 3)
(37, 56, 56, 3)
(38, 56, 56, 3)
(39, 56, 56, 3)
(40, 56, 56, 3)
(41, 56, 56, 3)
(42, 56, 56, 3)
(43, 56, 56, 3)
(44, 56, 56, 3)
(45, 56, 56, 3)
(46, 56, 56, 3)
(47, 56, 56, 3)
(48, 56, 56, 3)
(49, 56, 56, 3)
(50, 56, 56, 3)
(51, 56, 56, 3)
(52, 56, 56, 3)
(53, 56, 56, 3)
(54, 56, 56, 3)
(55, 56, 56, 3)
(56, 56, 56, 3)
(57, 56, 56, 3)
labeling class_name/kerasTes

error: /build/opencv-SviWsf/opencv-2.4.9.1+dfsg/modules/imgproc/src/imgwarp.cpp:1834: error: (-215) ssize.area() > 0 in function resize


In [2]:
import os
from PIL import Image
import PIL.ImageOps
# import urllib.parse # for python 3
from urlparse import urlparse # for python 2
from itertools import count
import random
import cv2
import numpy as np
import pandas as pd

"""
data/
    train/
        0/
            dog001.jpg
            dog002.jpg
            ...
        1/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        0/
            dog001.jpg
            dog002.jpg
            ...
        1/
            cat001.jpg
            cat002.jpg
            ...
"""

def make_label_for_each_directory(dir):
    """
    :param dir
    :return (big_array, label_array): (nparray, label)
    """

    big_array = []
    label_array = []
    for index, class_name in enumerate(os.listdir(dir), start=0):
        abs_path_class_name = dir + "/" + class_name
        if(os.path.isdir(abs_path_class_name)):
            print("labeling class_name"+'%s/%s'%(dir,str(class_name)) + "as " '%s'%index)
            #appending index to label_array 
            labelname = index
            for pic_name in os.listdir(abs_path_class_name):
                abs_path_pic_name = abs_path_class_name + "/" + pic_name
                if(verify(abs_path_pic_name)):
                    pic_array = read_image_as_array(abs_path_pic_name)
                    big_array.append(pic_array)
#                     print(np.asarray(big_array).shape)
                    label_array.append(labelname)
        else:
            print("not folder, %s" % class_name)
    #turn python array to numpy
    return (np.asarray(big_array), np.asarray(label_array))

def read_image_as_array(dir):
    """
    :param dir: directory of image name
    :return array
    """

    im = cv2.imread(dir)
    if im.shape[0] != 56 and im.shape[1] != 56:
        im = cv2.resize(im, (56, 56))
    im = im.tolist()
    return im

def data_generate(cur_path):
    """
    :param cur_path: input directory of train, validation data 
    :return (x_train, y_train), (x_test, y_test)
    """
    (x_train, y_train) = make_label_for_each_directory(cur_path + "/train")
    (x_test, y_test) = make_label_for_each_directory(cur_path + "/validation")
    return (x_train, y_train), (x_test, y_test)

def verify(img_name):
    """
    :param img_name: 
    :return Verify if img_name is img 
    """
    try:
        im=Image.open(img_name)
        return True
    except IOError:
        return False


data_directory = "data"
cur_path = os.path.dirname(os.path.abspath('__file__'))
(x_train, y_train), (x_test, y_test) = data_generate(cur_path + "/" + data_directory)


print("saving to csv")

df = pd.DataFrame(x_train)
df.to_csv("x_train"+".csv")

df = pd.DataFrame(y_train)
df.to_csv("y_train"+".csv")

df = pd.DataFrame(x_test)
df.to_csv("x_test"+".csv")

df = pd.DataFrame(y_test)
df.to_csv("y_test"+".csv")

print("finished saving to csv")


# The data, shuffled and split between train and test sets:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
# x_train shape: (50000, 32, 32, 3)
# y_train shape: (50000, 1)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

np.savetxt('test.csv', x_train, delimiter=',')
ndarr2 = np.loadtxt('test.csv', delimiter=',')
# Reference
# http://stackoverflow.com/questions/29839350/numpy-append-vs-python-append

not folder, .DS_Store
labeling class_name/kerasTest/data/train/0as 1
labeling class_name/kerasTest/data/train/1as 2
not folder, .DS_Store
labeling class_name/kerasTest/data/validation/0as 1
labeling class_name/kerasTest/data/validation/1as 2
saving to csv


ValueError: Must pass 2-d input

In [31]:
for i in ("x_train", "y_train", "x_test", "y_test"):
    print i

x_train
y_train
x_test
y_test
